In [54]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
from openpyxl.utils.exceptions import IllegalCharacterError
import re
import numpy as np

%load_ext autoreload
%autoreload 2

dir_path = r'C:\Users\fajar.fatoni\Documents\Python\Data\Zoho_Backup'
file_name = '\Leads_'

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In',
'GDPR Compliant',
'Website',
'Comments',
'Do you live in Bali?']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [56]:
path = r'c:\Users\fajar.fatoni\Documents\Python\Data\CRM_Rental_Report (2).xlsx'
# zoho_df = pd.read_csv(path, dtype='string')
zoho_df = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)
zoho_df['Mobile'] = 'empty'
zoho_df.drop(zoho_df.loc[zoho_df['Opt In'] == False])
zoho_df

KeyError: "['Booking_Id', 'TelNo', 'MobileNo', 'Email', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source', 'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname', 'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County', 'Country', 'Nationality', 'PostCode', 'Exclude From Mailing', 'Opt In', 'Booking Code', 'Booking Status', 'DateArrival', 'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code', 'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants', 'Contact Type', 'Odyssey Members', 'Mobile'] not found in axis"

# ====================== Awal Filter Umum ====================== 

In [ ]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'TelNo')
m.clean_number(zoho_df, 'Concat_Tel_Mob')
m.replace_not_valid_to_empty(zoho_df, 'TelNo')
m.replace_not_valid_to_empty(zoho_df, 'Concat_Tel_Mob')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:105: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

'Email'

In [ ]:
zoho_df.loc[
    (zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] != 'empty'), 
    'Phone'] = zoho_df['Mobile']
zoho_df.loc[zoho_df['Phone'] == zoho_df['Mobile'],'Mobile'] = 'empty'

In [ ]:
deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
display(deleted_wrong_format)
# deleted_wrong_format.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\deleted_wrong_format.xlsx', index=False)
count_deleted_wrong_format = deleted_wrong_format.shape[0]
count_after_deleted_wrong_format = zoho_df.shape[0]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile


In [ ]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['TelNo','Email', 'Concat_Tel_Mob'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['TelNo','Email', 'Concat_Tel_Mob'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile
233,zcrm_3584690000000461717,2019-01-09 11:11:23,empty,David,Rees,lucette@btinternet.com,441249818726,Calne,Wiltshire,United Kingdom,...,Past Guests,empty,Karma St. Martin's,"Other - Past Guests, Other Bali DB - Wifi, K...",True,True,False,2018-06-23 00:00:00,2018-06-30 00:00:00,empty
234,zcrm_3584690000000461718,2019-01-09 11:11:23,empty,David,Neville,lucette@btinternet.com,441249818726,empty,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-06-23 00:00:00,2018-06-30 00:00:00,empty
2279,zcrm_3584690000440835782,2023-04-17 16:20:03,empty,Tanya,Wilde,tanya@tastyadventures.co.uk,447818091113,empty,empty,United Kingdom,...,Past Guests,Viewpoint,Karma St. Martin's,empty,True,True,False,2023-04-09 00:00:00,2023-04-13 00:00:00,empty
2556,zcrm_3584690000461634397,2023-09-18 15:36:41,empty,Adrian,Eddleston,eddleston.adrian@gmail.com,447711705671,Grantham,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's;Karma St Martins,empty,True,True,False,2024-05-09 00:00:00,2024-12-09 00:00:00,empty
2983,zcrm_3584690000507753761,2024-09-09 12:01:58,empty,Melisssa,Bradbury,mellissab@me.com,447970756278,London,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2024-09-05 00:00:00,2024-09-07 00:00:00,empty
3094,zcrm_3584690000545825033,2025-05-09 14:59:18,empty,Jean,Gierlich,jeangierlich@gmail.com,17324854219,Williamsburg,empty,United States,...,Past Guests,empty,Karma St Martins,empty,True,False,False,2025-04-28 00:00:00,2025-04-30 00:00:00,empty


In [ ]:
zoho_df.drop_duplicates(subset=['TelNo','Email', 'Concat_Tel_Mob'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile
0,zcrm_3584690000000456111,2019-01-09 11:10:49,empty,Gwynfor,Davies,2gwyns@gmail.com,1443813058,Hengoed,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-08-10 00:00:00,2018-08-17 00:00:00,empty
1,zcrm_3584690000000456123,2019-01-09 11:10:49,empty,Trevor,Sherwood,44trevann@gmail.com,447828164927,empty,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-10-24 00:00:00,2018-10-26 00:00:00,empty
2,zcrm_3584690000000456144,2019-01-09 11:10:49,empty,Andrew,Latham,a.e.d.l@virginmedia.com,447758640150,Guildford,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-09-27 00:00:00,2018-09-28 00:00:00,empty
3,zcrm_3584690000000456146,2019-01-09 11:10:49,empty,Andrew,Halstead,a.halstead@btinternet.com,447811159329,London,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-07-25 00:00:00,2018-07-27 00:00:00,empty
4,zcrm_3584690000000456150,2019-01-09 11:10:49,empty,Aart,Wouterson,a.woutersen@planet.nl,31343412477,Dorn,empty,Netherlands,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-07-13 00:00:00,2018-07-15 00:00:00,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,zcrm_3584690000569060272,2025-09-08 14:51:36,empty,Alison,Crowter,alison@crowter.co.uk,447966828262,Dartmouth,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-08-30 00:00:00,2025-09-02 00:00:00,empty
3319,zcrm_3584690000569060302,2025-09-08 14:52:13,empty,Emelina,Elsden,emmelinastork@icloud.com,447780432399,Fittleworth,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-09-01 00:00:00,2025-09-05 00:00:00,empty
3320,zcrm_3584690000569066143,2025-09-08 14:51:28,empty,Jeff,Clements,geoffclemo45@gmail.com,441872572400,Truro,Goonhavern,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-08-29 00:00:00,2025-09-01 00:00:00,empty
3321,zcrm_3584690000569066233,2025-09-08 14:52:54,empty,Tba,Barker,c/opremierholidays@gmail.com,628444937888,empty,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-09-04 00:00:00,2025-09-06 00:00:00,empty


In [ ]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile


In [ ]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [ ]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [ ]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [ ]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile
295,zcrm_3584690000000463367,2019-01-09 11:11:33,empty,Peter,Dowling,pete.dowling@riverstouridb.org.uk,447729745806,Sittingbourne,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,"Other - Past Guests, Other Bali DB - Wifi, O...",True,True,False,2018-10-24 00:00:00,2018-10-26 00:00:00,empty


In [ ]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile
0,zcrm_3584690000000456111,2019-01-09 11:10:49,empty,Gwynfor,Davies,2gwyns@gmail.com,1443813058,Hengoed,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-08-10 00:00:00,2018-08-17 00:00:00,empty
1,zcrm_3584690000000456123,2019-01-09 11:10:49,empty,Trevor,Sherwood,44trevann@gmail.com,447828164927,empty,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-10-24 00:00:00,2018-10-26 00:00:00,empty
2,zcrm_3584690000000456144,2019-01-09 11:10:49,empty,Andrew,Latham,a.e.d.l@virginmedia.com,447758640150,Guildford,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-09-27 00:00:00,2018-09-28 00:00:00,empty
3,zcrm_3584690000000456146,2019-01-09 11:10:49,empty,Andrew,Halstead,a.halstead@btinternet.com,447811159329,London,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-07-25 00:00:00,2018-07-27 00:00:00,empty
4,zcrm_3584690000000456150,2019-01-09 11:10:49,empty,Aart,Wouterson,a.woutersen@planet.nl,31343412477,Dorn,empty,Netherlands,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-07-13 00:00:00,2018-07-15 00:00:00,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,zcrm_3584690000569060272,2025-09-08 14:51:36,empty,Alison,Crowter,alison@crowter.co.uk,447966828262,Dartmouth,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-08-30 00:00:00,2025-09-02 00:00:00,empty
3318,zcrm_3584690000569060302,2025-09-08 14:52:13,empty,Emelina,Elsden,emmelinastork@icloud.com,447780432399,Fittleworth,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-09-01 00:00:00,2025-09-05 00:00:00,empty
3319,zcrm_3584690000569066143,2025-09-08 14:51:28,empty,Jeff,Clements,geoffclemo45@gmail.com,441872572400,Truro,Goonhavern,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-08-29 00:00:00,2025-09-01 00:00:00,empty
3320,zcrm_3584690000569066233,2025-09-08 14:52:54,empty,Tba,Barker,c/opremierholidays@gmail.com,628444937888,empty,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-09-04 00:00:00,2025-09-06 00:00:00,empty


In [ ]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile
984,zcrm_3584690000006022375,2019-04-02 02:22:12,2017-08-30 00:00:00,Michael,Petri,noemail@rci.com,empty,Swanage,empty,United Kingdom,...,Past Guests,karma_st_martins,Karma St. Martin's,Other - Past Guests,True,True,False,empty,empty,empty


In [ ]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,Country,...,Lead Source,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile
0,zcrm_3584690000000456111,2019-01-09 11:10:49,empty,Gwynfor,Davies,2gwyns@gmail.com,1443813058,Hengoed,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-08-10 00:00:00,2018-08-17 00:00:00,empty
1,zcrm_3584690000000456123,2019-01-09 11:10:49,empty,Trevor,Sherwood,44trevann@gmail.com,447828164927,empty,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-10-24 00:00:00,2018-10-26 00:00:00,empty
2,zcrm_3584690000000456144,2019-01-09 11:10:49,empty,Andrew,Latham,a.e.d.l@virginmedia.com,447758640150,Guildford,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-09-27 00:00:00,2018-09-28 00:00:00,empty
3,zcrm_3584690000000456146,2019-01-09 11:10:49,empty,Andrew,Halstead,a.halstead@btinternet.com,447811159329,London,empty,United Kingdom,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-07-25 00:00:00,2018-07-27 00:00:00,empty
4,zcrm_3584690000000456150,2019-01-09 11:10:49,empty,Aart,Wouterson,a.woutersen@planet.nl,31343412477,Dorn,empty,Netherlands,...,Past Guests,empty,Karma St. Martin's,Other - Past Guests,True,True,False,2018-07-13 00:00:00,2018-07-15 00:00:00,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,zcrm_3584690000569060272,2025-09-08 14:51:36,empty,Alison,Crowter,alison@crowter.co.uk,447966828262,Dartmouth,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-08-30 00:00:00,2025-09-02 00:00:00,empty
3317,zcrm_3584690000569060302,2025-09-08 14:52:13,empty,Emelina,Elsden,emmelinastork@icloud.com,447780432399,Fittleworth,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-09-01 00:00:00,2025-09-05 00:00:00,empty
3318,zcrm_3584690000569066143,2025-09-08 14:51:28,empty,Jeff,Clements,geoffclemo45@gmail.com,441872572400,Truro,Goonhavern,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-08-29 00:00:00,2025-09-01 00:00:00,empty
3319,zcrm_3584690000569066233,2025-09-08 14:52:54,empty,Tba,Barker,c/opremierholidays@gmail.com,628444937888,empty,empty,United Kingdom,...,Past Guests,empty,Karma St Martins,empty,True,True,False,2025-09-04 00:00:00,2025-09-06 00:00:00,empty


In [ ]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['TelNo'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Concat_Tel_Mob'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Concat_Tel_Mob'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Concat_Tel_Mob'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [ ]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['TelNo'].isin(df_booking['Phone1']) | 
    zoho_df['TelNo'].isin(df_booking['Mobile']) |
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Phone1']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['TelNo'].isin(df_phone['Mobile']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['TelNo'].isin(df_booking['Phone1']) | 
    zoho_df['TelNo'].isin(df_booking['Mobile']) |
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Phone1']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['TelNo'].isin(df_phone['Mobile']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [ ]:

def categorize_contacts(df):
    conditions = [
        (df['Email'].to_numpy() != 'empty') & (df['TelNo'].to_numpy() == 'empty') & (df['Concat_Tel_Mob'].to_numpy() == 'empty'),
        (df['Email'].to_numpy() == 'empty') & ((df['TelNo'].to_numpy() != 'empty') | (df['Concat_Tel_Mob'].to_numpy() != 'empty')),
        (df['Email'].to_numpy() != 'empty') & ((df['TelNo'].to_numpy() != 'empty') | (df['Concat_Tel_Mob'].to_numpy() != 'empty'))
    ]
    choices = ['Email Only', 'Phone Only', 'Email and Phone']
    df['Contact Type'] = np.select(conditions, choices, default='Blank')

# Categorize contacts
categorize_contacts(zoho_df)

# Count contact types
count_email_only = (zoho_df['Contact Type'] == 'Email Only').sum()
count_phone_only = (zoho_df['Contact Type'] == 'Phone Only').sum()
count_email_and_phone = (zoho_df['Contact Type'] == 'Email and Phone').sum()
count_blank = (zoho_df['Contact Type'] == 'Blank').sum()

# Remove blank contacts if needed
zoho_df = zoho_df[zoho_df['Contact Type'] != 'Blank']
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [ ]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

np.int64(8)

In [ ]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Keseluruhan Master Zoho : "+str(count_data_awal))
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Keseluruhan Master Zoho : 5771
Jumlah Filter Leads Awal : 5771
Jumlah Format Email Salah : 0
Jumlah Setelah Clean Format Email Salah : 3329
Jumlah Duplicate (Phone, Mobile dan Email) : 6
Jumlah Setelah Clean Duplicate : 3323
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 3323
Jumlah Email OTA : 1
Jumlah Setelah Clean Email OTA : 3322
Jumlah Email Karma : 1
Jumlah Setelah Clean Email Karma : 3321
Jumlah Wash Member : 348
Jumlah Wash Booking : 2
Jumlah Setelah Wash Member : 2971
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 2971
Contact Type
Jumlah Email Only : 294
Jumlah Phone Only : 0
Jumlah Email dan Phone : 2677


In [ ]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 5771
Jumlah Wash Dedup : 8
Jumlah Wash Member : 348
Jumlah Wash Booking : 2
Jumlah Data Akhir : 2971


In [ ]:
zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\KSM - Past Guest Clean.xlsx', index=False)

# ====================== Akhir Filter Umum ====================== 

In [ ]:
# # Function to clean illegal characters except "Created Time"
# def clean_illegal_chars(df):
#     def remove_illegal_chars(value):
#         if isinstance(value, str):
#             return "".join(c for c in value if ord(c) >= 32)
#         return value
    
#     # Apply cleaning only to non-"Created Time" columns
#     for col in df.columns:
#         if col != "Created Time":  # Exclude "Created Time" from cleaning
#             df[col] = df[col].apply(remove_illegal_chars)

#     return df

# # Load your dataframe (zoho_df assumed to be already loaded)
# zoho_df.replace('empty', '', inplace=True, regex=True)

# # Format name columns properly
# zoho_df['First Name'] = zoho_df['First Name'].str.title()
# zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# # Define output path
# output_path = r'C:\Users\fajar.fatoni\Documents\Python\Data\kb.csv'

# try:
#     zoho_df.to_csv(output_path, index=False)
# except IllegalCharacterError:
#     print("IllegalCharacterError detected! Cleaning data...")
#     zoho_df = clean_illegal_chars(zoho_df)  # Clean only non-"Created Time" columns
#     zoho_df.to_csv(output_path, index=False)
#     print("File saved successfully after cleaning.")

# print(f"File saved at: {output_path}")


File saved at: C:\Users\fajar.fatoni\Documents\Python\Data\kb.csv


In [ ]:
zoho_df.groupby('Country').count()

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,City,State,BookRef,...,Lead Source Description,Lead Locations,Lead History,Opt In,GDPR Compliant,Agree with Terms and Condition,Arrival Date,Departure Date,Mobile,Contact Type
Country,,,,,,,,,,,,,,,,,,,,,
,146,146,146,146,146,146,146,146,146,146,...,146,146,146,146,146,146,146,146,146,146
Australia,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
Austria,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Belgium,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
Canada,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Czech Republic,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Denmark,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
England,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
Estonia,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
# zoho_df['Country'].loc[
#     (zoho_df['TelNo'].str.contains('^44|^\+44')) & 
#     (~zoho_df['Country'].str.contains('united kingdom|wales|england|scotland|northern ireland', regex=True, flags=re.I))
# ]